# Prueba Rappi

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

os.chdir(r"E:/Prueba Rappi/Prueba-Rappi/documentos")


In [3]:
dataset = pd.read_csv("Orders.csv")

# Convertir las fechas a datetime
dataset['created_at'] = pd.to_datetime(dataset['created_at'])


## 1. Análisis preliminar de los datos

En esta sección nos centraremos en analizar los datos de manera que nos permita dar una mejor visión y contexto de los registros trabajados. 

### Identificación de valores faltantes o valores nulos

In [4]:
dataset.isnull().sum()

order_id             0
store_id             0
to_user_distance     0
to_user_elevation    0
total_earning        0
created_at           0
taken                0
dtype: int64

In [5]:
dataset.isna().sum()

order_id             0
store_id             0
to_user_distance     0
to_user_elevation    0
total_earning        0
created_at           0
taken                0
dtype: int64

En este conjunto de datos, no se identifican valores faltantes o nulos.

### Estadística descriptiva básica

In [6]:
dataset.describe()

,order_id,store_id,to_user_distance,to_user_elevation,total_earning,taken
count,1.255490e+05,1.255490e+05,125549.000000,125549.000000,125549.000000,125549.000000
mean,1.507838e+07,7.485716e+08,1.476668,27.369828,5374.313360,0.922827
std,4.752629e+05,3.298381e+08,0.846750,84.818191,1784.919817,0.266867
min,1.421073e+07,3.000000e+07,0.000000,-260.333130,0.000000,0.000000
25%,1.467255e+07,9.000043e+08,0.809372,-18.417603,4200.000000,1.000000
50%,1.509367e+07,9.000077e+08,1.360573,11.781250,5200.000000,1.000000
75%,1.547552e+07,9.000127e+08,2.003069,62.978638,6400.000000,1.000000
max,1.590252e+07,9.000173e+08,9.360177,975.226074,108200.000000,1.000000


El promedio de ingreso de los couriers se encuentra en 5374 unidades monetarias del país de donde se toman los datos de operación, presumiblemente Colombia.

### Coeficiente de Variación.

Se usa para medir la variabilidad de un conjunto de datos dado. A pesar de que es una medida adimensional, es útil para relacionar la desviación estándar con el tamaño de la media. Se expresa como: 

$$C_v = \frac{\sigma}{\overline{x}}$$

In [7]:
# Coeficientes de variación para las variables numéricas
def coef_variacion(serie):
    return(round(np.std(serie) / np.mean(serie)*100, 2))

for i in [2,3,4]:
    print('Coeficiente de variación de {0}: {1}%'.format(dataset.columns[i], coef_variacion(dataset[dataset.columns[i]])))


Coeficiente de variación de to_user_distance: 57.34%
Coeficiente de variación de to_user_elevation: 309.9%
Coeficiente de variación de total_earning: 33.21%


En este caso, la elevación en metros entre la tienda y el usuario, es la variable con mayor variabilidad del conjunto de datos, por lo que será necesario reprocesar los datos antes de aplicar cualquier tipo de metodología estadística con el fin de identificar las causas de que una orden no sea aceptada.